In [ ]:
from src import eclipse_simulation_functions as esf
from src import trajectory_optimization_methods as tom
from src import plot_functions as pf
from src import data_loader_functions as dlf

import os
os.getcwd()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Reload all modules and create aliases for notebook
from importlib import reload
import shutil

# List of modules to reload
import src.eclipse_simulation_functions as esf
import src.trajectory_optimization_methods as tom
import src.plot_functions as pf

modules = [esf, tom, pf]

shutil.rmtree('src/__pycache__', ignore_errors=True)

for m in modules:
    reload(m)

G = tom.G
M_SUN = tom.M_SUN
M_EARTH = tom.M_EARTH
M_MOON = tom.M_MOON
a_Earth = tom.a_Earth
e_Earth = tom.e_Earth
a_Moon = tom.a_Moon
e_Moon = tom.e_Moon
DAY = tom.DAY
MONTH = tom.MONTH
YEAR = tom.YEAR
MASSES = tom.MASSES
T_FINAL = YEAR
DT = 15
t_array = np.arange(0, T_FINAL, DT)

M0  = 0


In [ ]:
# REAL DATA: MAY 01, 2022, 00:00 UTC - MAY 01, 2023, 00:00 UTC
# Source: NASA JPL Horizons

sun_path   = dlf.get_data_path("Sun_datas.txt")
earth_path = dlf.get_data_path("Earth_datas.txt")
moon_path  = dlf.get_data_path("Moon_datas.txt")

In [ ]:
# Load real data
t_nasa, history_nasa, v_nasa_history = dlf.load_horizons_history(sun_path, earth_path, moon_path)

In [ ]:
# Extracting initial positions, velocities: MAY 01, 2022, 00:00 UTC
r_sun_raw = history_nasa[0, 0]
r_earth_raw = history_nasa[0, 1]
r_moon_raw = history_nasa[0, 2]

v_sun_raw = v_nasa_history[0, 0]
v_earth_raw = v_nasa_history[0, 1]
v_moon_raw = v_nasa_history[0, 2]

In [ ]:
r_earth_vector_from_data = r_earth_raw - r_sun_raw
v_earth_vector_from_data = v_earth_raw - v_sun_raw

r_moon_rel_vector_from_data = r_moon_raw - r_earth_raw
v_moon_rel_vector_from_data = v_moon_raw - v_earth_raw

In [ ]:
moon_params = tom.rv_to_all_kepler_elements(r_moon_rel_vector_from_data, v_moon_rel_vector_from_data, G * M_EARTH)
a_moon_initial     = moon_params['a']
e_moon_initial     = moon_params['e']
i_moon_initial     = moon_params['i']
Omega_moon_initial = moon_params['Omega']
omega_moon_initial = moon_params['omega']
M0_moon_initial    = moon_params['M0']

earth_params = tom.rv_to_all_kepler_elements(r_earth_vector_from_data, v_earth_vector_from_data, G * M_SUN)

a_earth_initial     = earth_params['a']
e_earth_initial     = earth_params['e']
i_earth_initial     = earth_params['i']
Omega_earth_initial = earth_params['Omega']
omega_earth_initial = earth_params['omega']
M0_earth_initial    = earth_params['M0']


inclination_deg = np.degrees(i_moon_initial)

print(f"Calculated inclination: {inclination_deg:.2f} degrees")
if inclination_deg > 10:
    print("ERROR: The function still perceives the orbit in the equatorial coordinate system!")
else:
    print("SUCCESS: The function correctly detects the ecliptic plane.")

In [ ]:
hist_r_earth_kep_2d, hist_v_earth_kep_2d = esf.kepler_history(
    tom.solve_kepler_w_newton_backtrack_bisection,
    a_earth_initial,
    e_earth_initial,
    M_SUN,
    t_array,
    M0_earth_initial
)

hist_r_moon_kep_2d, hist_v_moon_kep_2d = esf.kepler_history(
    tom.solve_kepler_w_newton_backtrack_bisection,
    a_moon_initial,
    e_moon_initial,
    M_EARTH,
    t_array,
    M0_moon_initial
)


In [ ]:
hist_r_earth_kep_3d = esf.rotate_to_3d(hist_r_earth_kep_2d, 
                               i_earth_initial, Omega_earth_initial, omega_earth_initial)

hist_r_moon_kep_3d = esf.rotate_to_3d(hist_r_moon_kep_2d, 
                                 i_moon_initial, Omega_moon_initial, omega_moon_initial)

In [ ]:
r_earth_final_kep = hist_r_earth_kep_3d # On ecliptic
r_moon_final_kep  = hist_r_earth_kep_3d + hist_r_moon_kep_3d 

history_kepler = np.zeros((len(t_array), 3, 3))
history_kepler[:, 0, :] = 0.0
history_kepler[:, 1, :] = r_earth_final_kep
history_kepler[:, 2, :] = r_moon_final_kep

In [ ]:
r0_rel = r_moon_rel_vector_from_data 
v0_rel = v_moon_rel_vector_from_data

v_moon_mag = np.linalg.norm(v0_rel)
print(f"Moon initial velocity (relative): {v_moon_mag:.2f} m/s")

# The learning phase of the gradient descent method
T_MAX = 35 * DAY 
t_loss = np.linspace(0, T_MAX, 12)

In [ ]:
hist_r_raw, hist_v_raw = tom.velocity_verlet_3body_batch(
    v0_rel, 
    r0_rel, 
    r_earth_vector_from_data,
    v_earth_vector_from_data, 
    t_array, 
    save_history=True)

In [ ]:
# Create a temporary loss function that only waits for v_batch
def loss_wrapper(v_batch, r0_rel, t_loss):
    return tom.calculate_loss_batch_3body(v_batch, r0_rel, r_earth_vector_from_data, v_earth_vector_from_data, t_loss)

best_v = tom.gradient_nd(loss_wrapper, v0_rel, r0_rel, t_loss, niter=1000)
hist_r_opt, hist_v_opt = tom.velocity_verlet_3body_batch(
    best_v,           
    r0_rel,           
    r_earth_vector_from_data, 
    v_earth_vector_from_data,
    t_array,
    save_history=True
)

In [ ]:
print(f"Optimized relative v0: {best_v}")
print(f"Optimized relative v0 magnitude: {np.linalg.norm(best_v):.2f} m/s")

In [ ]:
history_verlet_raw = hist_r_raw[:, 0, :, :]
history_verlet_opt = hist_r_opt[:, 0, :, :]

In [ ]:
pf.print_period_std_dev(history_kepler, t_array, "Kepler")
pf.print_period_std_dev(history_verlet_raw, t_array, "Verlet (raw)")
pf.print_period_std_dev(history_verlet_opt, t_array, "Verlet (optimized)")
pf.print_period_std_dev(history_nasa, t_nasa, "NASA JPL Horizons")

In [ ]:
plt.figure(figsize=(7,7))

pf.plot_moon_geocentric(history_kepler, "Kepler", "#00ffe1", alpha=1.0, linewidth=0.7)
pf.plot_moon_geocentric(history_verlet_raw, "Verlet (raw)", "#39ff14", alpha=0.7, linewidth=0.7)
pf.plot_moon_geocentric(history_verlet_opt, "Verlet (optimal)", "#ffcc00", alpha=1.0, linewidth=0.7)
pf.plot_moon_geocentric(history_nasa, "NASA JPL Horizons", "#ec008c", alpha=0.5, linewidth=0.7)

plt.axis("equal")


plt.xlabel("x [km]")
plt.ylabel("y [km]")
plt.legend()
plt.title("Moon's orbit around Earth")
plt.show()



In [ ]:
plt.figure(figsize=(10,4))

pf.plot_distance_time(history_kepler, t_array, "Kepler", "#00ffe1")
pf.plot_distance_time(history_verlet_raw, t_array, "Verlet (raw)", "#39ff14")
pf.plot_distance_time(history_verlet_opt, t_array, "Verlet (optimized)", "#ffcc00")
pf.plot_distance_time(history_nasa, t_nasa, "NASA JPL Horizons", "#ec008c")

plt.xlabel("Time [day]")
plt.ylabel("Earth-Moon distance [km]")
plt.legend()
plt.title("Distance-Time")
plt.show()


In [ ]:
f = esf.get_double_mc_illumination
kepler_eclipses = esf.find_all_lunar_eclipses(history_kepler)
pf.plot_all_lunar_eclipses(f, history_kepler, t_array, kepler_eclipses, color="#00ffe1", label="Kepler", num_samples=8000)

verlet_raw_eclipses = esf.find_all_lunar_eclipses(history_verlet_raw)
pf.plot_all_lunar_eclipses(f, history_verlet_raw, t_array, verlet_raw_eclipses, color="#39ff14", label="Verlet (raw)", num_samples=8000)

verlet_opt_eclipses = esf.find_all_lunar_eclipses(history_verlet_opt)
pf.plot_all_lunar_eclipses(f, history_verlet_opt, t_array, verlet_opt_eclipses, color="#ffcc00", label="Verlet (optimized)", num_samples=8000)

nasa_eclipses = esf.find_all_lunar_eclipses(history_nasa)

if not nasa_eclipses:
    raise RuntimeError("No total lunar eclipse in Horizons data.")

# Refine the whole dataset based on all detected eclipses
t_refined_nasa, history_nasa_refined = pf.refine_all_eclipses(history_nasa, t_nasa, nasa_eclipses, factor=5, pad_factor=3.0)

# Re-run the detection on the smooth data to get precise eclipse timings
nasa_eclipses_refined = esf.find_all_lunar_eclipses(history_nasa_refined)
pf.plot_all_lunar_eclipses(f, history_nasa_refined, t_refined_nasa, nasa_eclipses_refined, color="#ec008c", label="NASA JPL Horizons", num_samples=8000)


In [ ]:
from pathlib import Path
# Save refined trajectories and eclipse info for fast reuse (3d_plot.py will load this)

out = Path("data/simulation_results.npz")
out.parent.mkdir(parents=True, exist_ok=True)

np.savez_compressed(
    out,
    h_kepler=history_kepler, t_kepler=t_array, e_kepler=np.array(kepler_eclipses, dtype=object), title_kepler="Kepler",
    h_verlet_raw=history_verlet_raw, t_verlet_raw=t_array, e_verlet_raw=np.array(verlet_raw_eclipses, dtype=object), title_verlet_raw="Verlet (raw)",
    h_verlet_opt=history_verlet_opt, t_verlet_opt=t_array, e_verlet_opt=np.array(verlet_opt_eclipses, dtype=object), title_verlet_opt="Verlet (opt)",
    h_nasa=history_nasa_refined, t_nasa=t_refined_nasa, e_nasa=np.array(nasa_eclipses_refined, dtype=object), title_nasa="NASA JPL Horizons"
)
print(f"Saved: {out}")

# Check for data loaded
import time
start = time.time()
with np.load(out, allow_pickle=True) as data:
    _ = data['h_nasa']
end = time.time()
print(f"Data loaded in: {end - start:.4f} seconds")